In [1]:
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if not IN_COLAB:
    os.chdir("..")
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")


In [4]:
train_df = pd.read_csv(r"C:\A\breastCancer\input/train.csv")
test_df = pd.read_csv(r"C:\A\breastCancer\input/test.csv")

In [8]:
train_df['PR_Allred_score'].unique()

array([ 6., nan,  4.,  8.,  2.,  7.,  5., 23., 54.,  3.])

In [10]:
train_df['PR_Allred_score'] =='23'

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: PR_Allred_score, Length: 1000, dtype: bool

In [ ]:


# CAT_COL fillna
train_df['NG']= train_df['NG'].fillna(2)
train_df['HG']= train_df['HG'].fillna(2)
train_df['HG_score_1']= train_df['HG_score_1'].fillna(3)
train_df['HG_score_2']= train_df['HG_score_2'].fillna(2)
train_df['HG_score_3']= train_df['HG_score_3'].fillna(1)
train_df['DCIS_or_LCIS_type']= train_df['DCIS_or_LCIS_type'].fillna(2)
train_df['T_category']= train_df['T_category'].fillna(1)
train_df['ER']= train_df['ER'].fillna(1)
train_df['PR']= train_df['PR'].fillna(1)
train_df['HER2']= train_df['HER2'].fillna(0)
train_df['HER2_IHC']= train_df['HER2_IHC'].fillna(2)
train_df['HER2_SISH']= train_df['HER2_SISH'].fillna(0)
train_df['BRCA_mutation']= train_df['BRCA_mutation'].fillna(0)

# NUL_COL fillna
# median
train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].median())
train_df['ER_Allred_score'] = train_df['ER_Allred_score'].fillna(train_df['ER_Allred_score'].median())
train_df['PR_Allred_score'] = train_df['PR_Allred_score'].fillna(train_df['PR_Allred_score'].median())
train_df['KI-67_LI_percent'] = train_df['KI-67_LI_percent'].fillna(train_df['KI-67_LI_percent'].median())
train_df['HER2_SISH_ratio'] = train_df['HER2_SISH_ratio'].fillna(train_df['HER2_SISH_ratio'].median())

# median
train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].median())
train_df = train_df.fillna(0)

test_df['암의 장경'] = test_df['암의 장경'].fillna(train_df['암의 장경'].median())
test_df = test_df.fillna(0)


target_name = ['N_category']
# 명목형 변수
cat_col_names = [
    '진단명','암의 위치', '암의 개수', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS_여부','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH',"BRCA_mutation"
]

for col_name in cat_col_names:
    cat_values = np.unique(train_df[col_name])
    
    #print(f"{col_name}: {cat_values}\n")


# 수치형 변수
num_col_names = [
    '나이','암의 장경','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio'
]

feature_columns = (
    num_col_names + cat_col_names + target_name)

train_df = train_df.loc[:,feature_columns]


test = test_df.loc[:,num_col_names + cat_col_names]


X = train_df.loc[:,num_col_names+cat_col_names]
y = train_df.loc[:,target_name]


In [ ]:
SEED = 3
NSPLIT = 10

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

In [ ]:
"""
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = SEED)
"""

from sklearn.model_selection import KFold

X = np.array(X)                 #np.array(df.iloc[:, :-1]) # class 열 제외한 feature 열들 모음 -> array 변환
y = np.array(y)                           #df['class']



In [ ]:
# split 개수, 셔플 여부 및 seed 설정
kf = KFold(n_splits = NSPLIT, shuffle = True, random_state = SEED)

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

##lightGBM

In [ ]:
import lightgbm as lgb

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    clf = lgb.LGBMClassifier(n_estimators=5, random_state=SEED) # 모델 선언
    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))
f1score = np.mean(f1_score_history)

accuracy 각 분할의 정확도 기록 : [0.74, 0.76, 0.76, 0.73, 0.78, 0.82, 0.81, 0.7, 0.8, 0.78]
accuracy 평균 정확도 : 0.768
f1_score 각 분할의 정확도 기록 : [0.75, 0.76, 0.7894736842105263, 0.7522935779816514, 0.8166666666666668, 0.8301886792452831, 0.8288288288288289, 0.7272727272727273, 0.7727272727272727, 0.78]
f1_score 평균 정확도 : 0.7807451436932957


## catboost classififier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = CatBoostClassifier(
      iterations=5, 
      learning_rate=0.1, 
      loss_function='CrossEntropy',
      random_state=SEED) # 모델 선언

    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

0:	learn: 0.6403783	total: 1.66ms	remaining: 6.63ms
1:	learn: 0.6025899	total: 3.71ms	remaining: 5.56ms
2:	learn: 0.5649702	total: 5.42ms	remaining: 3.61ms
3:	learn: 0.5410785	total: 7.09ms	remaining: 1.77ms
4:	learn: 0.5263394	total: 8.73ms	remaining: 0us
0:	learn: 0.6416820	total: 1.44ms	remaining: 5.76ms
1:	learn: 0.6054742	total: 3.04ms	remaining: 4.56ms
2:	learn: 0.5753061	total: 4.68ms	remaining: 3.12ms
3:	learn: 0.5509984	total: 6.38ms	remaining: 1.59ms
4:	learn: 0.5367123	total: 8.14ms	remaining: 0us
0:	learn: 0.6452672	total: 1.45ms	remaining: 5.8ms
1:	learn: 0.6095897	total: 3.29ms	remaining: 4.93ms
2:	learn: 0.5783693	total: 4.92ms	remaining: 3.28ms
3:	learn: 0.5531169	total: 6.76ms	remaining: 1.69ms
4:	learn: 0.5376801	total: 8.44ms	remaining: 0us
0:	learn: 0.6396706	total: 4.29ms	remaining: 17.2ms
1:	learn: 0.6036621	total: 5.94ms	remaining: 8.9ms
2:	learn: 0.5718673	total: 7.59ms	remaining: 5.06ms
3:	learn: 0.5465725	total: 9.38ms	remaining: 2.35ms
4:	learn: 0.5285892	tot

## test preidiction

In [ ]:
test_df_id = test_df.loc[:,'ID']
len(test_df_id),type(test_df_id)

(250, pandas.core.series.Series)

In [ ]:
array = np.array(test_df_id.values)
len(array),type(array)

(250, numpy.ndarray)

In [ ]:
test_pred = clf.predict(test)
len(test_pred),type(test_pred)

(250, numpy.ndarray)

In [ ]:
dic_pred={
    'ID' : array,
    'N_category':test_pred
}

In [ ]:
df_test_pred = pd.DataFrame(dic_pred)

In [ ]:
df_test_pred

,ID,N_category
0,BC_01_0011,1
1,BC_01_0220,1
2,BC_01_0233,0
3,BC_01_0258,1
4,BC_01_0260,1
...,...,...
245,BC_01_3328,1
246,BC_01_3404,0
247,BC_01_3418,0
248,BC_01_3438,0


In [ ]:
df_test_pred.to_csv(f'/content/drive/MyDrive/project/BreastCancer/ml/output/lgbm_{SEED}_{f1score}.csv')

In [ ]:
"""import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))


# print the scores on training and test set
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
f1score=f1_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
print('LightGBM Model f1score score: {0:0.4f}'.format(f1_score(y_pred, y_test)))"""

In [ ]:
"""from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    loss_function='CrossEntropy'
)
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))"""